In [3]:
#---------------------------------------------------------------START-----------------------------------------------------
# Source link: https://python.langchain.com/docs/tutorials/chatbot/

import os
import getpass

# Kerakli kalitlarni saqlash/Saving needed keys

In [4]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LangSmith uchun API kalitni kiriting")

LangSmith uchun API kalitni kiriting ········


In [5]:
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google Gemini uchun API kalitni kiriting")

Google Gemini uchun API kalitni kiriting ········


# Kerakli chat model bilan bo'glanish / Connecting to needed chat model

In [7]:
from langchain.chat_models import init_chat_model

try:
    model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
    print("Model bilan aloqa bog'landi")
except Exception as ex:
    print("Model bilan ulanishda muammo:", ex)

Model bilan aloqa bog'landi


In [9]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, I'm Azizbek")])

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


AIMessage(content='Hi Azizbek! Nice to meet you. How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--61b07c1c-e2fd-48b8-b9c1-912acf59dba9-0', usage_metadata={'input_tokens': 8, 'output_tokens': 48, 'total_tokens': 56, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 32}})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTP

# Modeldan ismimni so'rash / Asking my name from model 

In [12]:
model.invoke([HumanMessage(content="What's my name?")])
# input looks like: AIMessage(content="As an AI, I don't have access to personal information about you, including your name. I don't store user data or track individual identities.\n\n
#If you'd like me to call you something, please feel free to tell me!",...

AIMessage(content="As an AI, I don't have access to personal information about you, including your name. I don't retain details from past conversations or know who you are in the real world.\n\nSo, I don't know your name. If you'd like to tell me, feel free!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--281797b5-15e3-462e-8ec0-3e5677c5e121-0', usage_metadata={'input_tokens': 7, 'output_tokens': 786, 'total_tokens': 793, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 724}})

# Model bilan suhbatlarni modelga o'tkazish / Passing the conversatios to the model between human and AI

In [14]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, I'm Azizbek"),
        AIMessage(content="Hello Azizbek! How can I help you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Azizbek.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--577b88d5-b264-44ae-a0cd-e099a1d79462-0', usage_metadata={'input_tokens': 27, 'output_tokens': 53, 'total_tokens': 80, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 47}})

# Xabarning doimiyligi / Message persistence

In [17]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState,  StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {'messages' : response}

# Define the (single) node in the groph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [18]:
config = {"configurable" : {"thread_id" : "abs123"}}

In [26]:
query = "Hi! I'm Azizbek"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages" : input_messages}, config)
output["messages"][-1].pretty_print() # # output contains all messages in state


================================== Ai Message ==================================

Hi Azizbek! It's nice to meet you.

Now I know your name!


In [27]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages" : input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Azizbek!


### "thread_id" ni o'zgartirish / changing "thread_id"

In [28]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I still don't know your name. I'm an AI, and I don't have any personal information about you or access to your identity.

To know your name, you would have to tell me!


### Avvalgi holatga qaytarish / Going back the original conversation

In [29]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Azizbek.
